In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

#### 데이콘에서 제공하는 대회 데이터 활용
#####  - 대회명 : '도배 하자 질의 응답 처리 : 한솔데코 시즌2 AI 경진대회'
#####  - URL : https://dacon.io/competitions/official/236216/data

In [2]:
train_df = pd.read_csv('./llm_qna/data/train.csv')

In [3]:
train_df.head()

,id,질문_1,질문_2,category,답변_1,답변_2,답변_3,답변_4,답변_5
0,TRAIN_000,면진장치가 뭐야?,면진장치에 사용되는 주요 기술은 무엇인가요?,건축구조,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을...",면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
1,TRAIN_001,내진설계의 종류 좀 알려줘,내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요?,건축구조,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계에는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물이 지...","내진설계에는 주로 내진구조, 제진구조, 면진구조의 세 가지 종류가 있습니다. 이들은...","내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조...","내진 설계에는 다양한 종류가 있지만, 대표적으로 내진구조, 제진구조, 면진구조가 있..."
2,TRAIN_002,철골구조의 장점이 뭐야?,철골구조의 장점을 알려줘?,건축구조,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건...,철골구조의 장점은 건물의 외벽에 하중이 적게 걸리기 때문에 고층 건물의 건축이 용이...,"철골구조의 장점은 건물의 외벽이 하중이 걸리지 않아 공간 활용이 용이하고, 고층 건...",철골구조의 장점은 건물의 외벽에 하중이 크게 걸리지 않아 고층 건물을 건축할 수 있...
3,TRAIN_003,철골철근 콘크리트 구조가 뭐야?,철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?,건축구조,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","철골철근콘크리트 구조는 건축물을 지탱하는 주요 구조물인 철골과 철근, 그리고 콘크리...",철골철근 콘크리트 구조는 건축물을 지탱하기 위한 구조물에서 일반적으로 사용되는 방식...,"철골철근콘크리트 구조는 철골과 철근, 그리고 콘크리트를 함께 사용하여 만들어지는 건...","철골철근 콘크리트 구조는 강철 골조와 강철 철근, 그리고 콘크리트를 함께 사용하여 ..."
4,TRAIN_004,철골구조는 어떤 방식이 있어?,철골구조의 다양한 방식이 무엇인가요?,건축구조,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 일반철골구조와 경량철골구조가 있습니다. 일반철골구조는 주로 대형 건물이나...,철골구조는 주로 일반철골구조와 경량철골구조로 나뉘어집니다. 이들은 건축 시스템에 따...,철골구조는 주로 일반철골구조와 경량철골구조로 구분됩니다. 이외에도 최근에는 고층 건...,철골구조는 일반철골구조와 경량철골구조 두 가지 방식이 주로 사용됩니다. 일반철골구조...


In [24]:
# 질문 및 답변 데이터 각각 연결

Q_train_list = pd.concat([train_df['질문_1'], train_df['질문_2'],train_df['질문_1'], train_df['질문_2'], train_df['질문_2']]).tolist()
A_train_list = pd.concat([train_df['답변_1'], train_df['답변_2'],train_df['답변_3'], train_df['답변_4'], train_df['답변_5']]).tolist()

In [61]:
len(Q_train_list)

3220

In [25]:
type(Q_train_list)

list

In [11]:
# 한글을 제외한 숫자, 특수문자 제거
def prep_stc(sent):
    sent = re.sub(r"([^ㄱ-ㅎㅏ-ㅣ가-힣 ])",'', sent)
    return sent

In [27]:
print(prep_stc(Q_train_list[337]))

도배지 꼬임에 대해 설명해줘


In [28]:
print(prep_stc(A_train_list[337]))

도배지 꼬임 하자란 도배지에 꽈배기 형태의 주름이 생기는 것을 의미합니다 도배지 꼬임 하자가 발생하는 원인  책임소재  해결방법에 대해 설명드리겠습니다  공간 내 높은 습도 원인  습기가 도배지에 스며들거나 습기로 인해 도배지가 팽창할 때 꼬임이 발생할 수 있습니다 책임소재  건물의 소유자나 거주자가 습기 관리의 책임이 있습니다 해결 방법  제습기 가동 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업해야 합니다  벽면의손상불균형 원인  벽면에 손상이 있을 경우 손상된 벽을 통해 습기가 유입하는 등의 원인으로 인해 꼬임이 발생할 가능성이 있습니다 책임소재  벽면에 손상을 방생시킨 주체가 책임을 집니다 해결 방법  벽면을 제대로 복원하고 평평하게 균형을 맞춘 후 재작업 해야 합니다 해당 작업은 개인이 하기 어려우니 전문가의 도움을 받는 것을 추천합니다  불안정한 온도 원인  온도의 큰 변화는 도배지의 팽창과 수축을 유발하고 접착제의 안정성에도 영향을 미쳐 도배지 꼬임이 발생할 가능성이 있습니다 책임소재  거주자가 온도 관리에 책임이 있습니다 해결 방법  냉열기를 활용하여 적정온도를 유지해야하며 꼬임이 사라지지 않는다면 전문가의 도움을 받아 부분 혹은 전체 재작업을 해야 합니다  부적절한 접착제 사용 원인  도배에 부적절한 접착제를 사용함으로 인해 도배지 꼬임이 발생할 수 있습니다 책임소재  부적합한 접착제를 사용하여 시공한 시공자에게 책임이 있습니다 해결 방법  적절한 도배 접착제를 사용하여 재작업을 해야 합니다  건조시간 부족 원인  도배 후 완전 건조되기 전까지 꼬임이 발생할 수 있습니다 책임소재  없음 해결 방법  제조사 권장 건조시간일반적으로 일주일동안 기다려주세요 건조기간 동안에는 온도와 습도에 유의해야 합니다 일주일이 이후에도 꼬임이 있을 시 시공사에 연락해서 진단받아야 합니다  석고보드의 이동 원인  도배지를 붙인 석고보드가 이동하면서 꼬임이 발생할 수 있습니다 책임소재  석고보드 시공자 해결 방법  하자 부분의 도배지 제거 후 

In [30]:
sample_size = len(Q_train_list)

In [43]:
# 토큰화를 위하여 인코더 입력용 Q_tokens, 디코더 입력용 A_tokens_in, 디코더 출력용 A_tokens_out 리스트 생성

Q_token, A_token_in, A_token_out = [], [], []
Q_tokens, A_tokens_in, A_tokens_out = [], [], []

In [45]:
# 질문 및 답변 데이터의 토큰화 수행
# 디코더 입력 데이터에는 문장의 시작을 의미하는 '<sos>'
# 디코더 출력 데이터에는 문장의 끝을 의미하는 '<eos>' 삽입

for i in range(sample_size):

    Q_tokens.append(prep_stc(Q_train_list[i]).split())

    A_token_in = ['<sos>'] + prep_stc(A_train_list[i]).split()
    A_tokens_in.append(A_token_in)
    
    A_token_out = prep_stc(A_train_list[i]).split() + ['<eos>'] 
    A_tokens_out.append(A_token_out)

In [46]:
print(Q_tokens[337])
print(A_tokens_in[337])
print(A_tokens_out[337])

['도배지', '꼬임에', '대해', '설명해줘']
['<sos>', '도배지', '꼬임', '하자란', '도배지에', '꽈배기', '형태의', '주름이', '생기는', '것을', '의미합니다', '도배지', '꼬임', '하자가', '발생하는', '원인', '책임소재', '해결방법에', '대해', '설명드리겠습니다', '공간', '내', '높은', '습도', '원인', '습기가', '도배지에', '스며들거나', '습기로', '인해', '도배지가', '팽창할', '때', '꼬임이', '발생할', '수', '있습니다', '책임소재', '건물의', '소유자나', '거주자가', '습기', '관리의', '책임이', '있습니다', '해결', '방법', '제습기', '가동', '환기를', '통해', '실내', '적정', '습도를', '유지하고', '전문가의', '도움을', '받아', '보수작업해야', '합니다', '벽면의손상불균형', '원인', '벽면에', '손상이', '있을', '경우', '손상된', '벽을', '통해', '습기가', '유입하는', '등의', '원인으로', '인해', '꼬임이', '발생할', '가능성이', '있습니다', '책임소재', '벽면에', '손상을', '방생시킨', '주체가', '책임을', '집니다', '해결', '방법', '벽면을', '제대로', '복원하고', '평평하게', '균형을', '맞춘', '후', '재작업', '해야', '합니다', '해당', '작업은', '개인이', '하기', '어려우니', '전문가의', '도움을', '받는', '것을', '추천합니다', '불안정한', '온도', '원인', '온도의', '큰', '변화는', '도배지의', '팽창과', '수축을', '유발하고', '접착제의', '안정성에도', '영향을', '미쳐', '도배지', '꼬임이', '발생할', '가능성이', '있습니다', '책임소재', '거주자가', '온도', '관리에', '책임이', '있습니다', '해결', '방법', '냉열기를', '활용하여', '적정온도를', 

In [47]:
# 케라스 활용한 데이터 정수 인코딩 및 패딩 수행

Q_tkz = Tokenizer()
A_tkz = Tokenizer()

Q_tkz.fit_on_texts(Q_tokens)
encoder_input = Q_tkz.texts_to_sequences(Q_tokens)
padded_encoder_input = pad_sequences(encoder_input, padding = 'post')

A_tkz.fit_on_texts(A_tokens_in)
decoder_input = A_tkz.texts_to_sequences(A_tokens_in)
padded_decoder_input = pad_sequences(decoder_input, padding = 'post')

A_tkz.fit_on_texts(A_tokens_out)
decoder_output = A_tkz.texts_to_sequences(A_tokens_out)
padded_decoder_output = pad_sequences(decoder_output, padding = 'post')

In [48]:
print('인코더의 입력 크기(shape) :',padded_encoder_input.shape)
print('디코더의 입력 크기(shape) :',padded_decoder_input.shape)
print('디코더의 레이블 크기(shape) :',padded_decoder_output.shape)

인코더의 입력 크기(shape) : (3220, 22)
디코더의 입력 크기(shape) : (3220, 279)
디코더의 레이블 크기(shape) : (3220, 279)


In [49]:
padded_encoder_input[337]

array([  52, 1709,    9,   67,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0])

In [50]:
padded_decoder_input[337]

array([   2,   66, 2414, 5226,  170, 2990,  409, 2647,  615,   45,   74,
         66, 2414,  475,   64,  142,  146, 1297,  331,  662,  141,  339,
         37,  173,  142,  133,  170, 1993,  825,   19,  134, 2991,   25,
        257,   13,    3,    1,  146,   30,  250,  261,  193, 1042,  323,
          1,  143,  126,  948, 1298,  207,   17,   33,  267,  116,  288,
         62,   54,  186, 7497,    6, 7498,  142,  138,  771,  108,   15,
        872,  419,   17,  133, 2989,   32, 1838,   19,  257,   13,  224,
          1,  146,  138,  594, 6211, 5227, 1299, 1994,  143,  126,  213,
        256, 1708, 2648,  401, 1615,   39, 1995,  511,    6,  214,  294,
       1839,  444, 6212,   62,   54,  249,   45,  501, 5228,  181,  142,
        792,  144, 2196,  120, 1300, 1495, 2649, 1259, 2415,  102, 2992,
         66,  257,   13,  224,    1,  146,  261,  181,  595,  323,    1,
        143,  126, 3460,   21, 6213, 9792,  257, 1391,  470,   62,   54,
        186,  623,  716,  574,  681,  511,    6, 16

In [51]:
Q_vocab_size = len(Q_tkz.word_index) + 1
A_vocab_size = len(A_tkz.word_index) + 1
print("질문문장 단어 집합의 크기 : {:d}, 답변문장 단어 집합의 크기 : {:d}".format(Q_vocab_size, A_vocab_size))

질문문장 단어 집합의 크기 : 1906, 답변문장 단어 집합의 크기 : 16666


In [52]:
# 정수와 문자 간 입출력 변환을 위해 정수 -> 문자열, 문자열 -> 정수 딕셔너리 생성

Q_to_idx = Q_tkz.word_index
idx_to_Q = Q_tkz.index_word
A_to_idx = A_tkz.word_index
idx_to_A = A_tkz.index_word

In [58]:
# 테스트 데이터 분리를 위해 데이터 순서 섞어주기

indices = np.arange(padded_encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덤 시퀀스 :',indices)

padded_encoder_input = padded_encoder_input[indices]
padded_decoder_input = padded_decoder_input[indices]
padded_decoder_output = padded_decoder_output[indices]

랜덤 시퀀스 : [ 823  130 1982 ... 2961  573 1019]


In [59]:
padded_encoder_input[337]

array([ 240,   36, 1086,    1,   69,   43,   11,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0])

In [77]:
padded_decoder_input[337]

array([    2,  1590,    56,  2365,    12,   405, 13701,     3,     1,
         230,  1682,  4019,  4020,   451,   383,  2150,  1962,  3402,
          47,   564,     1,     4,  3353,  4855,  7171,  2552,  1704,
       13702,  2435,   313,  1379,  1704,    67,   809,     3,     1,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [80]:
idx_to_A[337]

'방법으로'

In [63]:
n_of_val = int(3320*0.1)
print('검증 데이터의 개수 :',n_of_val)

encoder_input_train = padded_encoder_input[:-n_of_val]
decoder_input_train = padded_decoder_input[:-n_of_val]
decoder_target_train = padded_decoder_output[:-n_of_val]

encoder_input_test = padded_encoder_input[-n_of_val:]
decoder_input_test = padded_decoder_input[-n_of_val:]
decoder_target_test = padded_decoder_output[-n_of_val:]

검증 데이터의 개수 : 332


In [64]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [65]:
embedding_dim = 64
hidden_units = 64

In [66]:
# 인코더
encoder_inputs = Input(shape=(None,))

enc_emb = Embedding(Q_vocab_size, embedding_dim)(encoder_inputs) # 임베딩 층

enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외

encoder_lstm = LSTM(hidden_units, return_state=True) # 상태값 리턴을 위해 return_state는 True

encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

In [67]:
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(A_vocab_size, hidden_units) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(A_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [69]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),\
          batch_size=128, epochs=50)

Epoch 1/50
23/23 [==============================] - 111s 5s/step - loss: 8.9274 - acc: 0.8494 - val_loss: 7.4769 - val_acc: 0.8560
Epoch 2/50
23/23 [==============================] - 112s 5s/step - loss: 5.7721 - acc: 0.8494 - val_loss: 3.8341 - val_acc: 0.8560
Epoch 3/50
23/23 [==============================] - 112s 5s/step - loss: 2.6678 - acc: 0.8494 - val_loss: 1.7781 - val_acc: 0.8560
Epoch 4/50
23/23 [==============================] - 114s 5s/step - loss: 1.6957 - acc: 0.8494 - val_loss: 1.5433 - val_acc: 0.8560
Epoch 5/50
23/23 [==============================] - 113s 5s/step - loss: 1.5243 - acc: 0.8494 - val_loss: 1.4153 - val_acc: 0.8560
Epoch 6/50
23/23 [==============================] - 114s 5s/step - loss: 1.4412 - acc: 0.8494 - val_loss: 1.3575 - val_acc: 0.8560
Epoch 7/50
23/23 [==============================] - 115s 5s/step - loss: 1.3808 - acc: 0.8494 - val_loss: 1.3023 - val_acc: 0.8560
Epoch 8/50
23/23 [==============================] - 116s 5s/step - loss: 1.3234 - a

In [81]:
model.save('llm_QnA_seq2seq')

INFO:tensorflow:Assets written to: llm_QnA_seq2seq\assets


INFO:tensorflow:Assets written to: llm_QnA_seq2seq\assets


In [70]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 설계 시작
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

# 수정된 디코더
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs2] + decoder_states2)

In [71]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
    states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = A_to_idx['<sos>']

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx_to_A[sampled_token_index]

        # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

    # <eos>에 도달하거나 정해진 길이를 넘으면 중단
    if (sampled_char == '<eos>' or
        len(decoded_sentence) > 100):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

    return decoded_sentence

In [72]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0):
            sentence = sentence + idx_to_Q[encoded_word] + ' '
    return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0 and encoded_word != A_to_idx['<sos>'] and encoded_word != A_to_idx['<eos>']):
            sentence = sentence + idx_to_A[encoded_word] + ' '
    return sentence

In [73]:
for seq_index in [3, 50, 100, 200]:
    input_seq = encoder_input_train[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)

    print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
    print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
    print("번역문장 :",decoded_sentence[1:-5])
    print("-"*50)

1/1 [==============================] - ETA: 0s

KeyboardInterrupt: 